In [10]:
!pip install neurokit2
!pip install wfdb
!pip install statsmodels==0.13.2
!pip install peakutils

In [11]:
!pip install pythoncrc

In [12]:
!pip install coverage>=4.4
!pip install pytest-cov
!pip install pyECG

In [3]:
import pandas as pd
import numpy as np
import numpy 
# Формат хранения данных
import h5py
from matplotlib import pyplot as plt
import seaborn as sns 
import pylab

# ско (среднеквадратическая ошибка) для оценки отклонения исходного сигнала от идеального
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Библиотеки для работы с сигналами
import scipy
import scipy.signal as signal
# Детекция пиков
import peakutils
import neurokit2 as nk

#!pip uninstall pyCRC
#!pip install pythoncrc
#from pyecg import ECGRecord
from pyecg import ECGRecord

from statsmodels.tsa.seasonal import STL 
from statsmodels.tsa.seasonal import seasonal_decompose
import wfdb
import mpl_toolkits
# Формат хранения данных
import math


In [15]:
def signalFromArray(arr):
  df= pd.DataFrame(arr)
  return df[1]

In [7]:

def FFA(data,dis):
  iSampleRate = dis					# Частота дискретизации, 1000 отсчетов в секунду
  x = data
  iSampleCount = x.shape[0]			# Номер образца
  t = np.linspace(0,iSampleCount/iSampleRate,iSampleCount)
  xFFT = np.abs(np.fft.rfft(x)/iSampleCount)  # Быстрое преобразование Фурье
  xFFTq=[]
  for i in xFFT:
    xFFTq.append(math.sqrt(i))
  xFreqs = np.linspace(0, iSampleRate/2, int(iSampleCount/2)+1).tolist()
  return xFreqs[5:20], xFFTq[5:20]

def locMax(df):
  maxi=[]
  for i in range(1,len(df)-1):
      if df[i] > df[i-1]:
          if df[i] > df[i+1]:
              maxi.append([df[i],0.5+i/10.0])
  return maxi
def getBaseFrequency(xFFTq):
  max=[-9999999,-1]
  for peak,aX in locMax(xFFTq):
    if max[0]<peak:
      max=[peak,aX]
  return max[1]


# устранение дрейф 
def removeSignalDrift(df,sampling_rate):
  xFreqs, xFFTq=FFA(df,sampling_rate)
  basePeriod = (int)(getBaseFrequency(xFFTq)**(-1)*sampling_rate)
  stlTrendSignal = STL(df,period=abs(basePeriod)).fit().trend
  return df-stlTrendSignal
# устранение помех
def removeHighFreq(x,w=10):
  avg=[]
  it=0
  while it+w<len(x):
    avg.append(x[it:it+w].sum()/w)
    it+=1
  return avg

In [8]:
sns.set_style('darkgrid')

pd.set_option('display.max_columns', 500)

pd.set_option("display.max_rows", None)


def plot_signal(ecg_sample, fsize=(15, 5)):
    plt.figure(figsize=fsize)
    plt.plot(ecg_sample)

def del_zspan(ecg_sample):
    del_fence = None
    for i in range(len(ecg_sample)-1, len(ecg_sample) // 2, -1):
        if ecg_sample[i] != 0:
            del_fence = i
            break
    return ecg_sample[: del_fence]

import os.path

def take_ecg_sample_xldb(person_idx, zone='II', sr=500, show=False):
    
    '''
    zone (lead-отведение):  ['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    sr (sample rate): 100Hz, 500Hz
    '''
    
    path = 'E:/диплом/ЭКГ Датасеты/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/records500'
    
    if sr == 100:
        path = 'E:/диплом/ЭКГ Датасеты/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/records100'
    
    if person_idx > 21837 or person_idx < 1:
        raise ValueError('Wrong index, 21837-last')
    
    len_pidx = abs(len(str(person_idx)) - 5)
    if len_pidx !=  0:
        str_idx = '0' * len_pidx
        str_idx += str(person_idx)
    else:
        str_idx = str(person_idx)
    
    

    str_fold = str_idx[:2] + "000"

    path += "/" + str_fold + "/" + str_idx
    
    if sr == 100:
        hea_path = path + '_lr.hea'
    else:
        hea_path = path + '_hr.hea'
    
    if os.path.exists(hea_path) != True :
      return "Nane"
    record = ECGRecord.from_wfdb(hea_path)


    time = record.time
    signal = numpy.array(record.get_lead(zone))
    
    #f show:
    #    plot_signal(signal)
    return signal

In [4]:
xl_df = pd.read_csv('E:/диплом/ЭКГ Датасеты/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/ptbxl_database.csv', index_col='ecg_id')
xl_df.head(2)

,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
ecg_id,,,,,,,,,,,,,,,,,,,,,
1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,...,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,...,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr


In [ ]:
#xl_df = pd.read_csv('/content/gdrive/MyDrive/ЭКГ/data/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/ptbxl_database.csv', index_col='ecg_id')
#xl_df.info()

In [37]:
def fun(ecg_data_path,Zone):
  df = pd.DataFrame()
  for i in Zone:
    ecg_signal= take_ecg_sample_xldb(ecg_data_path, zone=i, sr=500, show=False)
    ecg_signal = removeSignalDrift(ecg_signal,500)
    ecg_signal = removeHighFreq(ecg_signal,w=10)
    ecg_signal = pd.DataFrame(ecg_signal)
    df[i] = ecg_signal
  print(ecg_data_path)
  df.to_csv("E:/диплом/ЭКГ Датасеты/data500/"+str(ecg_data_path)+".csv")

Zone=['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
n = len(xl_df["patient_id"])
i=1 #5456 проблема
while i< 1000:
  fun(i,Zone)
  i=i+1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [39]:
Zone=['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
n = len(xl_df["patient_id"])
i=3000 #5456 проблема
while i< 4000:
  fun(i,Zone)
  i=i+1

3000
3001
3002
3003
3004
3005
3006
3007
3008
3009
3010
3011
3012
3013
3014
3015
3016
3017
3018
3019
3020
3021
3022
3023
3024
3025
3026
3027
3028
3029
3030
3031
3032
3033
3034
3035
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
3082
3083
3084
3085
3086
3087
3088
3089
3090
3091
3092
3093
3094
3095
3096
3097
3098
3099
3100
3101
3102
3103
3104
3105
3106
3107
3108
3109
3110
3111
3112
3113
3114
3115
3116
3117
3118
3119
3120
3121
3122
3123
3124
3125
3126
3127
3128
3129
3130
3131
3132
3133
3134
3135
3136
3137
3138
3139
3140
3141
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3157
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
3174
3175
3176
3177
3178
3179
3180
3181
3182
3183
3184
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
3197
3198
3199


In [40]:
Zone=['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
n = len(xl_df["patient_id"])
i=4480 #4480 проблема
while i< 5000:
  fun(i,Zone)
  i=i+1

4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4493
4494
4495
4496
4497
4498
4499
4500
4501
4502
4503
4504
4505
4506
4507
4508
4509
4510
4511
4512
4513
4514
4515
4516
4517
4518
4519
4520
4521
4522
4523
4524
4525
4526
4527
4528
4529
4530
4531
4532
4533
4534
4535
4536
4537
4538
4539
4540
4541
4542
4543
4544
4545
4546
4547
4548
4549
4550
4551
4552
4553
4554
4555
4556
4557
4558
4559
4560
4561
4562
4563
4564
4565
4566
4567
4568
4569
4570
4571
4572
4573
4574
4575
4576
4577
4578
4579
4580
4581
4582
4583
4584
4585
4586
4587
4588
4589
4590
4591
4592
4593
4594
4595
4596
4597
4598
4599
4600
4601
4602
4603
4604
4605
4606
4607
4608
4609
4610
4611
4612
4613
4614
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4630
4631
4632
4633
4634
4635
4636
4637
4638
4639
4640
4641
4642
4643
4644
4645
4646
4647
4648
4649
4650
4651
4652
4653
4654
4655
4656
4657
4658
4659
4660
4661
4662
4663
4664
4665
4666
4667
4668
4669
4670
4671
4672
4673
4674
4675
4676
4677
4678
4679


In [11]:
def fun1(ecg_data_path,Zone):
  df = pd.DataFrame()
  for i in Zone:
    ecg_signal= take_ecg_sample_xldb(ecg_data_path, zone=i, sr=500, show=False)
    ecg_signal = removeSignalDrift(ecg_signal,500)
    ecg_signal = removeHighFreq(ecg_signal,w=10)
    ecg_signal = pd.DataFrame(ecg_signal)
    df[i] = ecg_signal
  print(ecg_data_path)
  df.to_csv("E:/диплом/ЭКГ Датасеты/data500(II)/"+str(ecg_data_path)+".csv")

Zone=['II']
n = len(xl_df["patient_id"])
i=n #5456 проблема
while i< n+1:
  fun1(i,Zone)
  i=i+1

21837
